In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.neural_network import MLPClassifier


In [2]:
from sklearn.model_selection import train_test_split

In [8]:
dataTrain = pd.read_csv('../setTrainMap.csv')


In [9]:
dataTrain.head()

,PassengerId,Survived,Age,SibSp,Ticket,Parch,Fare,Sex_female,Pclass_3,Pclass_2,...,Embarked_S,Mr,Mrs,Miss,Master,Dr,OtherTitles,With_Cabin,Old,Kid
0,1,0,22.0,1,21171,0,7.2500,False,True,False,...,True,True,False,False,False,False,False,False,False,False
1,2,1,38.0,1,17599,0,71.2833,True,False,False,...,False,False,True,False,False,False,False,True,False,False
2,3,1,26.0,0,3101282,0,7.9250,True,True,False,...,True,False,False,True,False,False,False,False,False,False
3,4,1,35.0,1,113803,0,53.1000,True,False,False,...,True,False,True,False,False,False,False,True,False,False
4,5,0,35.0,0,373450,0,8.0500,False,True,False,...,True,True,False,False,False,False,False,False,False,False


In [10]:
dataTrain = dataTrain.drop("OtherTitles",axis=1)
dataTrain = dataTrain.drop("Dr",axis=1)
dataTrain = dataTrain.drop("Master",axis=1)
dataTrain = dataTrain.drop("PassengerId",axis=1)

In [15]:
dataPositivos = dataTrain[dataTrain['Survived'] == 1 ]

In [17]:
dataNegativos = dataTrain[dataTrain['Survived'] == 0 ].head(342)

In [22]:
dataTrainBalanceado = pd.concat([dataNegativos,dataPositivos])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(dataTrainBalanceado.drop(['Survived'], axis=1).values, dataTrainBalanceado["Survived"] , test_size=0.20, random_state=75)


In [24]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:

param= {"activation" : ["identity", "logistic", "tanh", "relu"],
        "solver" : ["lbfgs", "sgd", "adam"],
        "alpha" : [0.0001,0.001,0.01,0.1,1,10,100],
        "hidden_layer_sizes" : [(10,10),(10,10,10),(30,30),(30,30,30),(50,50),(50,50,50),(100,100),(100,100,100), (100,)],
       "learning_rate" : ["constant", "invscaling", "adaptive"],
        "max_iter": [100,200,300,400,500,600,700,800,900,1000]
       }

In [26]:
from sklearn.model_selection import RandomizedSearchCV
clf = MLPClassifier()
from sklearn.metrics import make_scorer, roc_auc_score,accuracy_score,f1_score
auc = make_scorer(accuracy_score)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = param, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring = auc)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_fun=15000, max_iter=200,
                                           momentum=0.9, n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5,
                                           random...
                                        'hidden_layer_sizes': [(10, 10),
                                                               (10, 10, 10),
                             

In [27]:
rf_random.best_params_

{'solver': 'adam',
 'max_iter': 900,
 'learning_rate': 'adaptive',
 'hidden_layer_sizes': (100, 100, 100),
 'alpha': 0.01,
 'activation': 'logistic'}

In [28]:
clf = MLPClassifier(activation= 'logistic'
, alpha= 0.01, solver= 'adam',hidden_layer_sizes= (100,100,100),
                   learning_rate = "adaptive", max_iter = 900)

In [29]:
clf.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=0.01, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100, 100), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=900,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [30]:
result = clf.predict(X_test)


In [31]:
from sklearn import metrics
pred = result
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.7931406384814496

In [32]:
(pred == y_test).mean()


0.7956204379562044